In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:


import numpy as np
import pandas as pd 
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

import spacy
nlp = spacy.load('en_core_web_sm')

from termcolor import colored
from wordcloud import WordCloud,STOPWORDS
from spacy import displacy
from nltk.tokenize import sent_tokenize, word_tokenize 

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('../input/nbme-score-clinical-patient-notes/train.csv')
test = pd.read_csv('../input/nbme-score-clinical-patient-notes/test.csv')
ss = pd.read_csv('../input/nbme-score-clinical-patient-notes/sample_submission.csv')
pn = pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')
features = pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv')
train = train.merge(features, on=['case_num','feature_num'], validate='m:1')
train = train.merge(pn, validate='m:1')
features.head()

In [ ]:
!pip install transformers
!pip download tokenizers==0.11.0


In [ ]:
import os

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
class CFG:
    wandb=True
    competition='NBME'
    _wandb_kernel='nakama'
    debug=False
    apex=True
    print_freq=100
    num_workers=4
    model="microsoft/deberta-base"
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=5
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=12
    fc_dropout=0.2
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]
# ====================================================
# wandb
# ====================================================
if CFG.wandb:
    
    import wandb

    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        secret_value_0 = user_secrets.get_secret("wandb_api")
        wandb.login(key=secret_value_0)
        anony = None
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')


    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(project='NBME-Public', 
                     name=CFG.model,
                     config=class2dict(CFG),
                     group=CFG.model,
                     job_type="train",
                     anonymous=anony)


In [ ]:
import numpy as np 
import pandas as pd 

train = pd.read_csv("../input/nbme-score-clinical-patient-notes/train.csv")
patient_notes = pd.read_csv("../input/nbme-score-clinical-patient-notes/patient_notes.csv")
best_answer_pn_num = train.query("`case_num` == 0 and `location` != '[]'").groupby(['pn_num']).size().idxmax()
# The following cell was copied was SANSKAR HASIJA's excellent EDA
PATIENT_IDX = best_answer_pn_num

patient_df = train[train['pn_num'] == PATIENT_IDX]
print(f"\033[94mPatient Notes - ")
print(f'\033[94m',patient_notes[patient_notes["pn_num"] == PATIENT_IDX]["pn_history"].iloc[0])
print("------------")
print(f'\033[92mAnnotaions:')
for i in range(len(patient_df)):
    print(f'\033[92m',patient_df["annotation"].iloc[i])

In [ ]:
best_answer_pn_num = train.query("`case_num` == 1 and `location` != '[]'").groupby(['pn_num']).size().idxmax()

# The following cell was copied was SANSKAR HASIJA's excellent EDA
PATIENT_IDX = best_answer_pn_num

patient_df = train[train['pn_num'] == PATIENT_IDX]
print(f"\033[94mPatient Notes - ")
print(f'\033[94m',patient_notes[patient_notes["pn_num"] == PATIENT_IDX]["pn_history"].iloc[0])
print("------------")
print(f'\033[92mAnnotaions:')
for i in range(len(patient_df)):
    print(f'\033[92m',patient_df["annotation"].iloc[i])

In [ ]:
train.query("`case_num` == 1")['feature_num'].unique()
random_id = train[train['case_num'] == 1].sample(random_state=12)['pn_num']
display(random_id.iloc[0])

patient_df = train[train['pn_num'] == random_id.iloc[0]]
print(f"\033[94mPatient Notes - ")
print(f'\033[94m',patient_notes[patient_notes["pn_num"] == PATIENT_IDX]["pn_history"].iloc[0])
print("------------")
print(f'\033[92mAnnotaions:')
for i in range(len(patient_df)):
    print(f'\033[92m',patient_df["annotation"].iloc[i])
#
best_answer_pn_num = train.query("`case_num` == 2 and `location` != '[]'").groupby(['pn_num']).size().idxmax()

# The following cell was copied was SANSKAR HASIJA's excellent EDA
PATIENT_IDX = best_answer_pn_num

patient_df = train[train['pn_num'] == PATIENT_IDX]
print(f"\033[94mPatient Notes - ")
print(f'\033[94m',patient_notes[patient_notes["pn_num"] == PATIENT_IDX]["pn_history"].iloc[0])
print("------------")
print(f'\033[92mAnnotaions:')
for i in range(len(patient_df)):
    print(f'\033[92m',patient_df["annotation"].iloc[i])

In [ ]:
import numpy as np 
import pandas as pd
from tqdm.auto import tqdm
import random
import os
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import dill
import tensorflow.keras.backend as K
from tqdm.auto import tqdm
from tensorflow.keras import mixed_precision
import matplotlib.pyplot as plt
import tensorflow as tf
from transformers import AutoTokenizer, AutoConfig,TFAutoModel
import json
import gc
import string
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
%env TOKENIZERS_PARALLELISM=true


In [ ]:
try: # detect TPUs
    tpu  = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    tf.config.experimental_connect_to_cluster(tpu )
    tf.tpu.experimental.initialize_tpu_system(tpu )
    strategy = tf.distribute.TPUStrategy(tpu )
    print('Using TPU')
except ValueError: # detect GPUs
    tpu = None
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)


In [ ]:
seed= 59
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
TRAIN = False 

In [ ]:
features = pd.read_csv("../input/nbme-score-clinical-patient-notes/features.csv")
patient_notes = pd.read_csv("../input/nbme-score-clinical-patient-notes/patient_notes.csv")
test = pd.read_csv("../input/nbme-score-clinical-patient-notes/test.csv")
train= pd.read_csv("../input/nbme-score-clinical-patient-notes/train.csv")
sample_submission= pd.read_csv("../input/nbme-score-clinical-patient-notes/sample_submission.csv")

In [ ]:
features.loc[27, 'feature_text'] = 'Last-Pap-smear-1-year-ago'

In [ ]:
train.loc[338, 'location'] = ('[[764 783]]')

train.loc[621, 'location'] = ('[[77 100]]')

train.loc[655, 'location'] = ('[[285 292;301 312], [285 287;296 312]]')

train.loc[1262, 'location'] = ('[[551 557;565 580]]')

train.loc[1265, 'location'] = ('[[131 135;181 212]]')

train.loc[1396, 'location'] = ('[[259 280]]')

train.loc[1591, 'location'] = ('[[176 184;201 212]]')

train.loc[1615, 'location'] = ('[[249 257;271 288]]')

train.loc[1664, 'location'] = ('[[822 824;907 924]]')

train.loc[1714, 'location'] = ('[[101 129]]')

train.loc[1929, 'location'] = ('[[531 539;549 561]]')

train.loc[2134, 'location'] = ('[[540 560;581 593]]')

train.loc[2191, 'location'] = ('[[32 57]]')

train.loc[2553, 'location'] = ('[[308 317;376 384]]')

train.loc[3124, 'location'] = ('[[549 557]]')

train.loc[3858, 'location'] = ('[[102 123], [102 112;125 141], [102 112;143 157], [102 112;159 171]]')

train.loc[4373, 'location'] = ('[[33 45]]')

train.loc[4763, 'location'] = ('[[5 16]]')

train.loc[4782, 'location'] = ('[[175 194]]')

train.loc[4908, 'location'] = ('[[700 723]]')

train.loc[6016, 'location'] = ('[[225 250]]')

train.loc[6192, 'location'] = ('[[197 218;236 260]]')

train.loc[6380, 'location'] = ('[[480 482;507 519], [480 482;499 503;512 519], [480 482;521 531], [480 482;533 545], [480 482;564 582]]')

train.loc[6562, 'location'] = ('[[290 320;327 337], [290 320;342 358]]')

train.loc[6862, 'location'] = ('[[288 296;324 363]]')

train.loc[7022, 'location'] = ('[[108 182]]')

train.loc[7422, 'location'] = ('[[102 121]]')

train.loc[8876, 'location'] = ('[[481 483;533 552]]')

train.loc[9027, 'location'] = ('[[92 102], [123 164]]')

train.loc[9938, 'location'] = ('[[89 117], [122 138], [368 402]]')

train.loc[9973, 'location'] = ('[[344 361]]')

train.loc[10513, 'location'] = ('[[600 611], [607 623]]')

train.loc[11551, 'location'] = ('[[386 400;443 461]]')

train.loc[11677, 'location'] = ('[[160 201]]')

train.loc[12124, 'location'] = ('[[325 337;349 366]]')

train.loc[12279, 'location'] = ('[[405 459;488 524]]')

train.loc[12289, 'location'] = ('[[353 400;488 524]]')

train.loc[13238, 'location'] = ('[[293 307], [321 331]]')

train.loc[13297, 'location'] = ('[[182 221], [182 213;225 234]]')

train.loc[13299, 'location'] = ('[[79 88], [409 418]]')

train.loc[13845, 'location'] = ('[[86 94;230 236], [86 94;237 256]]')

train.loc[14083, 'location'] = ('[[56 64;156 179]]')

In [ ]:
test = test.merge(patient_notes,on=['case_num','pn_num']).merge(features,on=['case_num','feature_num'])
train = train.merge(patient_notes,on=['case_num','pn_num']).merge(features,on=['case_num','feature_num'])

In [ ]:
MODEL_NAME = "bert-base-uncased"
DATA_PATH = "../input/nbmeqatfdata"
DATA_EXISTS = os.path.exists(DATA_PATH)
MODEL_NAME

In [ ]:
if DATA_EXISTS and TRAIN:
    ! cp -r ../input/nbmeqatfdata/my_tokenizer .
    ! cp ../input/nbmeqatfdata/*.dill .
normalization = True

In [ ]:
if DATA_EXISTS:
    tokenizer = AutoTokenizer.from_pretrained(DATA_PATH+"/my_tokenizer/",normalization=normalization)
    config = AutoConfig.from_pretrained(DATA_PATH+"/my_tokenizer/config.json")
else:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,normalization=normalization)
    config = AutoConfig.from_pretrained(MODEL_NAME)
    tokenizer.save_pretrained('my_tokenizer')
    config.save_pretrained('my_tokenizer')

In [ ]:
FEATURES = features.feature_num.unique().tolist()
N_FEATURES = len(FEATURES)
SEQUENCE_LENGTH = 512
def decode_location(locations):
    for x in ["[","]","'"]:
        locations = locations.replace(x,'')
    locations = locations.replace(',',';')
    locations = locations.split(";")
    res = []
    for location in locations:
        if location:
            x,y = location.split()
            res.append((int(x),int(y)))
    return sorted(res,key=lambda x:x[0])

def build_data():
    input_ids_arr,attention_mask_arr,token_type_ids_arr,answers =[],[],[],[]
    feature_ids ,case_ids = [],[]
    for g1 in tqdm(train.groupby('pn_num')):
        gdf = g1[1]
        pn_history  = gdf.iloc[0].pn_history

        for index, row in gdf.iterrows():
            question = row.feature_text
            answer_empty = True
            tokens = tokenizer.encode_plus(pn_history,question, max_length=SEQUENCE_LENGTH, padding='max_length',truncation=True, return_offsets_mapping=True)
            input_ids = np.array(tokens['input_ids'],dtype=np.int32)
            attention_mask = np.array(tokens['attention_mask'],dtype=np.uint8)
            token_type_ids = np.array(tokens['token_type_ids'],dtype=np.uint8)
            offsets = tokens['offset_mapping']
            answer_mask = np.zeros(SEQUENCE_LENGTH,dtype=np.uint8)
            # Answer mask
            for i, (w_start, w_end) in enumerate(offsets):
                for start,end in decode_location(row.location):
                    if w_start < w_end and (w_start >= start) and (end >= w_end):
                        answer_mask[i] = 1
                        answer_empty = False
                    if w_start >= w_end:
                        break
            if not answer_empty:
                input_ids_arr.append(input_ids)
                attention_mask_arr.append(attention_mask)
                token_type_ids_arr.append(token_type_ids)
                answers.append(answer_mask)
                feature_ids.append(row.feature_num)
                case_ids.append(row.case_num)
    input_ids_arr = np.array(input_ids_arr,dtype=np.int32)
    attention_mask_arr = np.array(attention_mask_arr,dtype=np.uint8)
    token_type_ids_arr = np.array(token_type_ids_arr,dtype=np.uint8)
    answers = np.array(answers,dtype=np.uint8)
    feature_ids = np.array(feature_ids,dtype=np.int32)
    case_ids = np.array(case_ids,dtype=np.int32)
    return feature_ids,case_ids,input_ids_arr,attention_mask_arr,token_type_ids_arr,answers
if DATA_EXISTS:
    data = dill.load(open(DATA_PATH+"/data.dill",'rb'))
else:
    data = build_data()
    dill.dump(data,open('data.dill','wb'))

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:
def to_dataset(data,batch_size=32 if tpu else 12,shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices(data).map(lambda a,b,c,d:((a,b,c),d))
    size = len(ds)
    steps = size//batch_size
    ds = ds.repeat()
    if shuffle:
        ds = ds.shuffle(size)
    ds = ds.batch(batch_size).prefetch(buffer_size=AUTO)
    return ds,steps
n_splits = 5
cv = GroupKFold(n_splits=n_splits)
feature_ids,case_ids,input_ids_arr,attention_mask_arr,token_type_ids_arr,answers = data
data_splits = []
for n, (train_index, val_index) in enumerate(cv.split(feature_ids , case_ids,feature_ids)):
    train_data = input_ids_arr[train_index],attention_mask_arr[train_index],token_type_ids_arr[train_index],answers[train_index]
    test_data = input_ids_arr[val_index],attention_mask_arr[val_index],token_type_ids_arr[val_index],answers[val_index]
    data_splits.append((to_dataset(train_data),to_dataset(test_data,shuffle=False)))
    del train_data,test_data

In [ ]:
def build_model():
    
    tokens = tf.keras.layers.Input(shape=(SEQUENCE_LENGTH,), name = 'tokens', dtype=tf.int32)
    attention = tf.keras.layers.Input(shape=(SEQUENCE_LENGTH,), name = 'attention', dtype=tf.int32)
    token_type_id = tf.keras.layers.Input(shape=(SEQUENCE_LENGTH,), name = 'token_type_id', dtype=tf.int32)
    
    if not TRAIN:
        config = AutoConfig.from_pretrained(DATA_PATH+"/my_tokenizer/config.json")
        backbone = TFAutoModel.from_config(config)
    else:
        print(f"Loading {MODEL_NAME}...")
        config = AutoConfig.from_pretrained(MODEL_NAME)
        backbone = TFAutoModel.from_pretrained(MODEL_NAME,config=config)
        
    out = backbone(tokens, attention_mask=attention,token_type_ids=token_type_id)[0]
    
    out = tf.keras.layers.Dropout(0.2)(out)
    out = tf.keras.layers.Dense(1, activation='sigmoid')(out)
    
    model = tf.keras.Model([tokens,attention,token_type_id],out)
    
    return model

In [ ]:
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
epochs = 50
init_lr = 1e-5

In [ ]:
if TRAIN:
    val_key = "val_f1_m"
    scores = []
    with strategy.scope():
        i = 0
        for (train_ds,steps_per_epoch),(test_ds,steps) in data_splits:
            model = build_model()
            callback = tf.keras.callbacks.EarlyStopping(monitor=val_key,mode='max', patience=20)
            ckp_callback = tf.keras.callbacks.ModelCheckpoint(
                                                    filepath=f'model{i}.h5',
                                                    save_weights_only=True,
                                                    monitor=val_key,
                                                    mode='max',
                                                    options=tf.train.CheckpointOptions(experimental_io_device='/job:localhost'),
                                                    save_best_only=True)
            reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor=val_key,mode='max',factor=0.2,patience=2, min_lr=1e-6)
            callbacks=[callback,ckp_callback,reduce_lr]
            # Compile the model
            model.compile(optimizer=tf.keras.optimizers.Adam(init_lr),
                          loss=tf.keras.losses.BinaryCrossentropy(),
                          metrics=['acc',f1_m])

            history = model.fit(train_ds,
                                steps_per_epoch=steps_per_epoch,
                                validation_data=test_ds,
                                validation_steps=steps,
                                epochs=epochs,
                                callbacks=callbacks)
            scores.append(max(history.history[val_key]))
            i += 1
if TRAIN:
    print(scores)
    print(f"CV Score : {np.mean(scores)}")

In [ ]:
def build_test():
    input_ids, attention_masks, token_type_ids,offsets,row_ids = [],[],[],[],[]
    for g1 in tqdm(test.groupby('pn_num')):
        gdf = g1[1]
        for index, row in gdf.iterrows():
            pn_history = row.pn_history
            question = row.feature_text
            row_id = row.id
            tokens = tokenizer.encode_plus(pn_history, question, max_length=SEQUENCE_LENGTH, padding='max_length',
                                           truncation=True, return_offsets_mapping=True)
            input_id = np.array(tokens['input_ids'], dtype=np.int32)
            attention_mask = np.array(tokens['attention_mask'], dtype=np.uint8)
            token_type_id = np.array(tokens['token_type_ids'], dtype=np.uint8)
            offset = tokens['offset_mapping']

            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            token_type_ids.append(token_type_id)
            offsets.append(offset)
            row_ids.append(row_id)
            del question,row_id,tokens,input_id,attention_mask,token_type_id,offset

    attention_masks = np.array(attention_masks, dtype=np.uint8)
    token_type_ids = np.array(token_type_ids, dtype=np.uint8)
    input_ids = np.array(input_ids, dtype=np.int32)
    return row_ids,offsets,input_ids,attention_masks,token_type_ids
row_ids,offsets,input_ids,attention_masks,token_type_ids = build_test()

In [ ]:
model = build_model()
path =  DATA_PATH if not TRAIN else "."
preds = None 
for i in range(n_splits):
    print(f"SPLIT {i}")
    model.load_weights(path+f"/model{i}.h5")
    pred = model.predict((input_ids,attention_masks,token_type_ids),batch_size=16)
    if preds is None:
        preds = pred
    else: 
        preds += pred
    del pred
    gc.collect()
preds = preds/(i+1)

In [ ]:
all_special_ids = set(tokenizer.all_special_ids)

In [ ]:
def decode_position(pos):
    return ";".join([" ".join(np.array(p).astype(str)) for p in pos])


def translate(preds,row_ids,input_ids,offsets,token_type_ids):
    all_ids = []
    all_pos = []

    for k in range(len(preds)):
        offset = offsets[k]
        pred = preds[k]
        row_id = row_ids[k]
        input_id = input_ids[k]
        token_type_id = token_type_ids[k]
        prediction = []
        pred = (pred>0.5).astype(np.uint8)
        
        i = 0
        while i<SEQUENCE_LENGTH:
            
            if token_type_id[i] != 0:
                break
            if int(input_id[i]) in all_special_ids:
                i += 1
                continue
            if pred[i] == 0:
                i += 1
                continue
            if pred[i] == 1:
                start = min(offset[i])
                while i<SEQUENCE_LENGTH:
                    if pred[i] != 1:
                        break
                    elif token_type_id[i] != 0:
                        break
                    elif int(input_id[i]) in all_special_ids:
                        break
                    else:
                        end = max(offset[i])
                    i += 1
                if  end == 0:
                    break
                prediction.append((start,end))
            else:
                i+=1
        all_ids.append(row_id)
        all_pos.append(decode_position(prediction))
            
    df = pd.DataFrame({
        "id":all_ids,
        "location": all_pos
    })
    return df

**submission**

In [ ]:
sub = translate(preds,row_ids,input_ids,offsets,token_type_ids)
sub.to_csv('submission.csv',index=False)
sub.head(5)

In [ ]:
train.sort_values(by="id").reset_index()[["id","location"]].head(10)

In [ ]:
sample_submission= pd.read_csv("../input/nbme-score-clinical-patient-notes/sample_submission.csv")
def build_test():
    input_ids, attention_masks, token_type_ids,offsets,row_ids = [],[],[],[],[]
    for g1 in tqdm(test.groupby('pn_num')):
        gdf = g1[1]
        for index, row in gdf.iterrows():
            pn_history = row.pn_history
            question = row.feature_text
            row_id = row.id
            tokens = tokenizer.encode_plus(pn_history, question, max_length=SEQUENCE_LENGTH, padding='max_length',
                                           truncation=True, return_offsets_mapping=True)
            input_id = np.array(tokens['input_ids'], dtype=np.int32)
            attention_mask = np.array(tokens['attention_mask'], dtype=np.uint8)
            token_type_id = np.array(tokens['token_type_ids'], dtype=np.uint8)
            offset = tokens['offset_mapping']

            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            token_type_ids.append(token_type_id)
            offsets.append(offset)
            row_ids.append(row_id)
            del question,row_id,tokens,input_id,attention_mask,token_type_id,offset

    attention_masks = np.array(attention_masks, dtype=np.uint8)
    token_type_ids = np.array(token_type_ids, dtype=np.uint8)
    input_ids = np.array(input_ids, dtype=np.int32)
    return row_ids,offsets,input_ids,attention_masks,token_type_ids
row_ids,offsets,input_ids,attention_masks,token_type_ids = build_test()
model = build_model()
path =  DATA_PATH if not TRAIN else "."
preds = None 
for i in range(n_splits):
    print(f"SPLIT {i}")
    model.load_weights(path+f"/model{i}.h5")
    pred = model.predict((input_ids,attention_masks,token_type_ids),batch_size=16)
    if preds is None:
        preds = pred
    else: 
        preds += pred
    del pred
    gc.collect()
preds = preds/(i+1)
all_special_ids = set(tokenizer.all_special_ids)

In [ ]:
sub = translate(preds,row_ids,input_ids,offsets,token_type_ids)
sub.to_csv('submission.csv',index=False)
sub.head(5)